### 1. GBM의 개요 및 실습

> __부스팅__ 
- 여러개의 약한 학습기를 __순차적으로 학습, 예측__ 하면서 잘못 예측한 데이터에 __가중치 부여__ 를 통해 오류를 개선해 나가면서 학습하는 방식
- 대표적인 부스팅 방식으로 __AdaBoost__ 와 __Gradient Boost__ 가 있음  

>__Gradient Descent__ (경사하강법)
- GBM은 AdaBoost와 유사하나 가중치 업데이트를 Gradient Descent를 이용한다는 점에서 차이가 있음
- 경사하강법은 반복 수행을 통해 오류를 최소화 할 수 있도록 가중치의 업데이트 값을 도출하는 기법을 의미

> __GBM의 장단점__
- __장점__ : __과적합__ 에 강한 알고리즘
- __단점__ : 약한 학습기의 __순차적인 예측오류 보정__ 을 통해 학습을 수행하기 때문에 병렬처리가 지원 되지 않음 (n_jobs=-1 사용 불가) -> 수행시간이 오래걸림

In [1]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data = old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] > 0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset():
    ## 각 데이터 파일은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('data/human_activity/features.txt', sep = '\s+', header = None, names = ['column_index', 'column_name'])
    
    ## 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 dfm 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    ## dfm에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    ## 학습 피처 데이터세트와 테스트 피처 데이터를 dfm으로 로딩. 칼럼명은 feature_name 적용
    X_train = pd.read_csv('data/human_activity/train/X_train.txt', sep='\s+', names = feature_name)
    X_test = pd.read_csv('data/human_activity/test/X_test.txt', sep='\s+', names = feature_name)
    
    ## 학습 레이블과 테스트 레이블 데이터를 dfm으로 로딩하고 칼럼명은 action으로 부여
    y_train = pd.read_csv('data/human_activity/train/y_train.txt', sep='\s+', names = ['action'])
    y_test = pd.read_csv('data/human_activity/test/y_test.txt', sep='\s+', names = ['action'])

    ## 로드된 학습/테스트용 dfm을 모두 반환
    return X_train, X_test, y_train, y_test

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings 
import pandas as pd
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

## GBM 수행 시간 측정을 위해 시작 시간 설정
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, y_train)
pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, pred)

print(f'GBM 정확도 : {gb_accuracy:.4f}')
print(f'GBM 수행시간 : {time.time() - start_time:.1f}초')

GBM 정확도 : 0.9382
GBM 수행시간 : 682.8초


### 2. GBM 하이퍼 파라미터 튜닝

> __주요 하이퍼 파라미터__  
- GBM도 트리 기반의 알고리즘이므로 n_estimators, max_depth 등 기본적인 하이퍼 파라미터는 동일
- __loss__ : 경사 하강법에 사용할 비용 함수 지정. 기본겂은 'deviance'
- __learning_rate__ : GBM이 학습을 진행할 떄마다 적용하는 학습률(회귀에서 자세한 내용을 다룸) 0 ~ 1사이 값을 지정할 수 있으며 디폴트는 0.1, 작을수록 예측 성능이 높아질 가능성이 높지만 수행시간이 오래걸림
- __n_estimators__ : 약한 학습기의 갯수 디폴트는 100, 클 수록 성능이 좋아질 가능성이 있음
- __subsample__ : 약한 학습기가 학습에 사용하는 데이터의 샘플링 비율. 기본값은 1, 과적합이 염려될 경우 1보다 작은 값으로 설정

In [3]:
from sklearn.model_selection import GridSearchCV

start_time = time.time()
params = {
    'n_estimators' : [100, 500],
    'learning_rate' : [0.05, 0.1]
}

grid_cv = GridSearchCV(gb_clf, param_grid = params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print(f'최고 예측 정확도: {grid_cv.best_score_:.4f}')
print(f'GBM 수행시간 : {time.time() - start_time:.1f}초')

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 66.7min finished


최적 하이퍼 파라미터:
 {'learning_rate': 0.05, 'n_estimators': 500}
최고 예측 정확도: 0.9013
GBM 수행시간 : 6480.7초


In [7]:
## GridSearchCV를 이용해 최적으로 학습된 estimator로 예측 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print(f'GBM 정확도: {gb_accuracy:.4f}')

GBM 정확도: 0.9396
